In [ ]:
-- Question 1.

INSERT INTO player_stats_daily (date, player_id, wins, losses, win_rate)
SELECT 
	CAST(datetime AS date) AS date,
	player_id,
	SUM(CASE WHEN result = "Win" THEN 1
			 ELSE 0
		END) AS wins,
	SUM(CASE WHEN result = "Loss" THEN 1
			 ELSE 0
		END) AS losses,
	SUM(CASE WHEN result = "Win" THEN 1 ELSE 0 END)/COUNT(result) AS win_rate
FROM matches
WHERE date = "2018-06-17"
GROUP BY player_id

In [ ]:
-- Question 2a.

SELECT *
FROM matches m
WHERE datetime IN (
	SELECT MAX(datetime)
	FROM matches
	WHERE player_id = m.player_id
)
GROUP BY m.player_id

In [ ]:
-- Question 2b.
-- Don't think I did this one right, I think I need to add another nested query in the WHERE clause with the 5 most recent dates?

SELECT player_id, SUM(CASE WHEN result = "Win" THEN 1 ELSE 0 END)/5 AS win_rate
FROM matches
WHERE player_id IN (
    SELECT player_id
    FROM matches
    GROUP BY player_id
    HAVING COUNT(result) >= 5
)
GROUP BY player_id
ORDER BY datetime DESC
LIMIT 5

In [ ]:
-- Question 3.

SELECT player_id, SUM(spend) AS total_spend
FROM (
	SELECT *
	FROM purchases_a 
	UNION 
	SELECT *
	FROM purchases_b
)
GROUP BY player_id

In [ ]:
-- Question 4a.

SELECT SUM(wins)/(SUM(wins) + SUM(losses)) AS win_rate_veterans
FROM player_stats_daily
WHERE player_id IN (
	SELECT player_id
	FROM player_stats_daily
	WHERE date < "2018-06-17"
)

In [ ]:
-- Question 4b.

SELECT
	CASE WHEN player_id IN (
			SELECT player_id
			FROM player_stats_daily
			WHERE date < "2018-06-17"
		) THEN "Veteran Players"
		ELSE "New Players"
	END AS player_group,
	SUM(wins) OVER (PARTITION BY player_group)/(SUM(wins) OVER (PARTITION BY player_group) + SUM(losses) OVER (PARTITION BY player_group)) AS win_rate
FROM player_stats_daily